# Evaluate the Effect of Training 
We can assess whether retraining Difflinker leads to improved performance in two ways:
1. Evaluate how much the success rate improves with re-training
2. The difference between the total number of stable MOFs found w/ and w/o a closed loop

In [9]:
from itertools import chain
from scipy.interpolate import interp1d
from pathlib import Path
import pandas as pd

## Route 1: Measure Success Rate by Model Generation

## Round 2: Assess workflow outcomes w/o retraining
Show that it gets better

### Get the "Stable Found" at 90 minutes
Loop over all runs and store: scale, if retrained or not, and the number of stable found after 90 minutes. 
The 450-node run switches how it trained DiffLinker at around 90 minutes, and we don't want to study that effect yet.

In [33]:
hours = 1.5

In [34]:
success_data = []
for path in chain(Path('summaries').glob('*-nodes.csv.gz'), Path('summaries').glob('*-nodes_repeat-*.csv.gz'), Path('summaries').glob('*no-retrain*.csv.gz')):
    # Get metadata
    count = int(path.name.split("-")[0])
    retrain = 'no-retrain' not in path.name

    # Pull the success rate
    mofs = pd.read_csv(path)
    num_found = interp1d(mofs['walltime'], mofs['cumulative_found'], kind='previous')(hours * 3600).item()

    success_data.append({
        'nodes': count,
        'retrain': retrain,
        'found': num_found,
        'found_node-hr': num_found / (count * hours)
    })
success_data = pd.DataFrame(success_data)

In [35]:
success_data.groupby(['nodes', 'retrain']).mean()

found  found_node-hr
nodes retrain                       
32    False     133.0       2.770833
      True      313.0       6.520833
64    False     426.5       4.442708
      True      641.0       6.677083
128   True     1622.0       8.447917
256   True     3633.0       9.460938
450   True     6554.0       9.709630

TBD: Make a plot